In [1]:
import pandas as pd
import numpy as np
import os
import re
import json

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Gapminder World Data set



## Source files

In [337]:
# indicator name and their hash names. The hashs are used as file name in 
# indicator json folder waffle-server-importers-exporters-world-legacy-with-data/data/out/gw/indicators/
idt = pd.read_json('./source/quantities.json')

# groupings, there are 5 ways of grouping of countries in the data set.
gps = pd.read_json('./source/area_categorizarion.json')

# Country properities
geo = pd.read_excel('./source/country_synonyms.xlsx')

# Region properities
reg_f = open('./source/regions.json')
regs = json.load(reg_f)

# indicator properities
concepts = pd.read_csv('./source/graph_settings - Indicators.csv')

# old en.json, to be updated
enj = json.load(open('./source/en.json'))

# dont-panic-poverty data points
dpp = pd.read_csv('./source/dont-panic-poverty.csv')

# discrete concept list from systema_globalis. 
# we only use this file to get the world_4region concept
sgdc = pd.read_csv('./source/ddf--concepts--discrete.csv')

# country properities from systema_globalis
geo_sg = pd.read_csv('./source/ddf--entities--geo--country.csv')


## helper function

In [21]:
def to_concept_id(s, sub='[/ -\.\*";]+', sep='_'):
    '''convert a string to lowercase alphanumeric + underscore id for concepts'''
    
    if s is np.nan:
        return s
    
    s1 = re.sub(sub, sep, s.strip())
#     s1 = re.sub(r'\[.*\]', '', s1)
    s1 = s1.replace('\n', '')

    if s1[-1] == sep:  # remove the last underscore
        s1 = s1[:-1]

    return s1.lower()


In [22]:
def rename_col(s):
    '''map concept name to concept id'''
    real = idt[idt['-t-ind'] == s]['-t-name'].iloc[0]
    cid = concepts[concepts['name'] == real]['concept'].iloc[0]
#     print(real, ': ', cid)
    
    return cid

In [23]:
def rename_geo(s):
    """map gwid to iso code"""
    return geo_.get_value(s, 'ISO3dig_ext')

In [24]:
def to_dict_dropna(data):
  return dict((k, v.dropna().to_dict()) for k, v in data.iterrows())

# Entities of country gourps

In [77]:
regd = {}  # a dictionary, which keys are region id and values are region names

for i in regs:
    regd[i.get(list(i.keys())[0])] = list(i.keys())[0]

In [548]:
for i, n in gps.n.apply(to_concept_id).iteritems():
    df = pd.DataFrame([], columns=[n, 'name', 'gwid', 'is--'+n])
    df['gwid'] = gps.iloc[i]['groupings'].keys()
    if i == 4:
        df[n] = df['gwid'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+', sep=''))
    else:
        df[n] = df['gwid'].apply(lambda x: to_concept_id(regd[x], sub='[/ -\.\*";\[\]]+'))

    df['name'] = df['gwid'].apply(lambda x: regd[x])
    df['is--'+n] = 'TRUE'

    df.to_csv('./output/ddf/ddf--entities--geo--'+n+'.csv', index=False)

# Entities of Country

In [74]:
geo_ = geo[['ISO3dig_ext', 'Gwid']]
geo_ = geo_.set_index('Gwid')
geo_2 = geo.set_index('Gwid').drop('ISO3dig_ext', axis=1)

In [78]:
country = geo_.copy()

for i, n in gps.n.apply(to_concept_id).iteritems():

    res = {}

    for k, v in gps.iloc[i]['groupings'].items():
    #     print(regd[k])
        for c in v:
            if c:
#                 if c == 'i144':
#                     print(k, regd[k])
                if i == 4:
                    res[c] = to_concept_id(regd[k], sub='[/ -\.\*";\[\]]+', sep='')
                else:
                    res[c] = to_concept_id(regd[k], sub='[/ -\.\*";\[\]]+')

    ser = pd.Series(res)
    
    country[n] = ser

In [533]:
geo

,Use Name,Gapminder list,Alternative 1,Alternative 2,Alternative 3,Alternative 4 (CDIAC),pandg,GOD_ID,ISO3dig_ext,ALT 5,NAME,CODE,Number,Gwid,arb1,arb2,arb3,arb4,arb5,arb6
0,Abkhazia,Abkhazia,NaN,NaN,NaN,NaN,NaN,GE-AB,_abk,NaN,NaN,NaN,NaN,i0,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Afghanistan,Islamic Republic of Afghanistan,NaN,NaN,Afghanistan,AFGHANISTAN,AF,afg,NaN,AFGHANISTAN,AFG,4,i1,NaN,NaN,NaN,NaN,NaN,NaN
2,Akrotiri and Dhekelia,Akrotiri and Dhekelia,NaN,NaN,NaN,NaN,NaN,Akrotiri_Dhekelia,_akr,NaN,NaN,NaN,NaN,i2,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,Albania,NaN,NaN,NaN,Albania,ALBANIA,AL,alb,NaN,ALBANIA,ALB,8,i3,NaN,NaN,NaN,NaN,NaN,NaN
4,Algeria,Algeria,NaN,NaN,NaN,Algeria,ALGERIA,DZ,dza,NaN,ALGERIA,DZA,12,i4,NaN,NaN,NaN,NaN,NaN,NaN
5,American Samoa,American Samoa,NaN,NaN,NaN,NaN,NaN,AS,asm,NaN,AMERICAN SAMOA,ASM,16,i5,NaN,NaN,NaN,NaN,NaN,NaN
6,Andorra,Andorra,NaN,NaN,NaN,NaN,NaN,AD,and,NaN,ANDORRA,AND,20,i6,NaN,NaN,NaN,NaN,NaN,NaN
7,Angola,Angola,NaN,NaN,NaN,Angola,ANGOLA,AO,ago,NaN,ANGOLA,AGO,24,i7,NaN,NaN,NaN,NaN,NaN,NaN
8,Anguilla,Anguilla,NaN,NaN,NaN,NaN,NaN,AI,aia,NaN,ANGUILLA,AIA,660,i8,NaN,NaN,NaN,NaN,NaN,NaN
9,Antigua and Barbuda,Antigua and Barbuda,NaN,NaN,NaN,Antigua & Barbuda,NaN,AG,atg,NaN,ANTIGUA AND BARBUDA,ATG,28,i9,NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
country2 = pd.concat([country, geo_2], axis=1)

In [291]:
country2 = country2.reset_index()
country2 = country2.rename(columns={'NAME': 'Upper Case Name', 'Use Name': 'Name', 'ISO3dig_ext': 'country_2'})
country2.columns = list(map(to_concept_id, country2.columns))
country2['is--country'] = 'TRUE'

In [81]:
# country2 = country2.set_index('country').rename(index=m)  #MODI
# country2 = country2.reset_index()

In [292]:
country3 = pd.read_csv('./source/ddf--entities--geo--country.csv', encoding='latin')  # TODO: add a issue about United Korea (former)

In [293]:
country3 = country3[['geo', 'world_4region', 'latitude', 'longitude', 'name']]

In [294]:
country3 = country3.rename(columns={'geo': 'country'}).set_index('name')

In [295]:
country4 = pd.concat([country2.set_index('name'), country3], axis=1)

In [296]:
country4 = country4.reset_index()
country4 = country4.rename(columns={'index': 'name'})

In [176]:
ccs = country4.columns

In [177]:
ccs

Index(['name', 'country', 'gwid', 'geographic_regions', 'income_groups',
       'landlocked', 'g77_and_oecd_countries',
       'geographic_regions_in_4_colors', 'main_religion_2008',
       'gapminder_list', 'alternative_1', 'alternative_2', 'alternative_3',
       'alternative_4_cdiac', 'pandg', 'god_id', 'alt_5', 'upper_case_name',
       'code', 'number', 'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6',
       'is--country', 'country_2', 'world_4region', 'latitude', 'longitude'],
      dtype='object')

In [297]:
c2c = ['country', 'gwid', 'name', 'alt_5', 'alternative_1', 'alternative_2', 'alternative_3',
       'alternative_4_cdiac', 'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6',
       'code', 'g77_and_oecd_countries', 'gapminder_list',
       'geographic_regions', 'geographic_regions_in_4_colors', 'god_id',
        'income_groups', 'is--country', 'landlocked', 'latitude',
       'longitude', 'main_religion_2008', 'number', 'pandg',
       'upper_case_name', 'world_4region']

country4.loc[:, c2c].to_csv('./output/ddf/ddf--entities--geo--country.csv', index=False)

In [298]:
country4.head()

,name,gwid,country_2,geographic_regions,income_groups,landlocked,g77_and_oecd_countries,geographic_regions_in_4_colors,main_religion_2008,gapminder_list,...,arb2,arb3,arb4,arb5,arb6,is--country,country,world_4region,latitude,longitude
0,Abkhazia,i0,_abk,europe_central_asia,NaN,NaN,others,europe,NaN,Abkhazia,...,NaN,NaN,NaN,NaN,NaN,TRUE,abkh,europe,NaN,NaN
1,Afghanistan,i1,afg,south_asia,low_income,landlocked,g77,asia,muslim,Afghanistan,...,NaN,NaN,NaN,NaN,NaN,TRUE,afg,asia,33,66
2,Akrotiri and Dhekelia,i2,_akr,europe_central_asia,NaN,coastline,others,europe,NaN,Akrotiri and Dhekelia,...,NaN,NaN,NaN,NaN,NaN,TRUE,akr_a_dhe,europe,NaN,NaN
3,Albania,i3,alb,europe_central_asia,upper_middle_income,coastline,others,europe,muslim,Albania,...,NaN,NaN,NaN,NaN,NaN,TRUE,alb,europe,41,20
4,Algeria,i4,dza,middle_east_north_africa,upper_middle_income,coastline,g77,africa,muslim,Algeria,...,NaN,NaN,NaN,NaN,NaN,TRUE,dza,africa,28,3


# Concepts

In [18]:
# TODO: because concept source file changed, need to change below too.

In [338]:
dsc = concepts.columns

In [339]:
dsc

Index(['Name', 'Tooltip', 'Menu level1', 'Menu level 2', 'Indicator url',
       'Download', 'ddf_id', 'Scale', 'ddf_name', 'ddf_unit', 'old_ddf_id',
       'interpolation'],
      dtype='object')

In [340]:
concepts.columns = list(map(to_concept_id, concepts.columns))

In [341]:
concepts['concept_type'] = 'measure'

In [342]:
concepts = concepts.drop(['download', 'old_ddf_id'], axis=1)

In [343]:
concepts = concepts.iloc[:, [5, 0, 1,2,3,4,6,7, 8, 9, 10]]

In [344]:
concepts = concepts.rename(columns={'ddf_id':'concept', 'name': 'full_name','ddf_name':'name', 'ddf_unit': 'unit', 'tooltip': 'description'})

In [345]:
concepts.head()

,concept,full_name,tooltip,menu_level1,menu_level_2,indicator_url,scale,name,unit,interpolation,concept_type
0,children_per_woman_total_fertility,Children per woman (total fertility),Total fertility rate. The number of children t...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,Babies per woman,NaN,NaN,measure
1,co2_emissions_tonnes_per_person,CO2 emissions (tonnes per person),Carbon dioxide emissions from the burning of f...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,CO2 emissions,tons per person,NaN,measure
2,income_per_person_gdppercapita_ppp_inflation_a...,"Income per person (GDP/capita, PPP$ inflation-...",Gross domestic product per person adjusted for...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,log,Income per person,$/year (GDP/capita),NaN,measure
3,child_mortality_0_5_year_olds_dying_per_1000_born,"Child mortality (0-5 year-olds dying per 1,000...",Death of children under five years of age per ...,NaN,NaN,http://spreadsheets.google.com/pub?key=0ArfEDs...,log,Child mortality rate,deaths under age 5 per 1000 births,NaN,measure
4,life_expectancy_years,Life expectancy (years),The average number of years a newborn child wo...,NaN,NaN,http://spreadsheets.google.com/pub?key=phAwcNA...,lin,Life expectancy,years,NaN,measure


In [90]:
dsc.drop('ddf_id')

Index(['Name', 'Tooltip', 'Menu level1', 'Menu level 2', 'Indicator url',
       'Download', 'Scale'],
      dtype='object')

In [35]:
len(concepts)

544

In [346]:
cc = concepts[['concept', 'name', 'concept_type', 'description', 'indicator_url', 'scale', 'unit', 'interpolation']].copy()

In [15]:
# cc.columns = [['concept', 'name', 'concept_type', 'tooltip', 'indicator_url']]

In [432]:
k = concepts[concepts.concept == u'———————————————————————'].index

In [348]:
cc = cc.drop(k)

In [349]:
cc['drill_up'] = np.nan
cc['domain'] = np.nan

In [350]:
cc['scales'] = cc['scale'].apply(lambda x: ['log', 'linear'] if x=='log' else ['linear', 'log'])

In [352]:
dc = pd.DataFrame([], columns=cc.columns)

In [353]:
dcl = list(map(to_concept_id, gps.n.values))

In [354]:
ccs = ['gapminder_list', 'alternative_1', 'alternative_2', 'alternative_3',
       'alternative_4_cdiac', 'pandg', 'god_id', 'alt_5', 'upper_case_name',
       'code', 'number', 'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6',
       'world_4region', 'latitude', 'longitude']

In [355]:
ccs_id = list(map(to_concept_id, ccs))

In [356]:
dcl_ = np.r_[dcl, ['geo', 'country','time', 'name', 'gwid', 'name_short', 'name_long', 'description'], 
             [x+'_id' for x in dcl], ccs_id, ['indicator_url', 'scale', 'unit', 'interpolation']]


dcl_2 = np.r_[gps.n.values, ['Geo', 'Country','Time', 'Name', 'Gwid', 'Name Short', 'Name Long', 'Description'], 
              [x+' id' for x in gps.n.values], ccs, ['indicator_url', 'Scale', 'Unit', 'interpolation']]

In [357]:
dc['concept'] = dcl_
dc['name'] = dcl_2

In [358]:
dc['concept_type'] = 'string'
dc.loc[:5, 'concept_type'] = 'entity_set'
dc.loc[6, 'concept_type'] = 'entity_domain'
dc.loc[7, 'concept_type'] = 'entity_set'
dc.loc[8, 'concept_type'] = 'time'
dc.loc[37, 'concept_type'] = 'entity_set'
dc.loc[[38,39], 'concept_type'] = 'measure'
dc.loc[[38,39], 'unit'] = 'degrees'
dc.loc[38, 'scale'] = 'lat'
dc.loc[39, 'scale'] = 'long'

In [359]:
# dc['concept_type'] = 'string'
dc.loc[:5, 'domain'] = 'geo'

dc.loc[7, 'drill_up'] = dcl
dc.loc[7, 'domain'] = 'geo'
dc.loc[37, 'domain'] = 'geo'

In [360]:
dc

,concept,name,concept_type,tooltip,indicator_url,scale,unit,interpolation,drill_up,domain,scales
0,geographic_regions,Geographic regions,entity_set,NaN,NaN,NaN,NaN,NaN,NaN,geo,NaN
1,income_groups,Income groups,entity_set,NaN,NaN,NaN,NaN,NaN,NaN,geo,NaN
2,landlocked,Landlocked,entity_set,NaN,NaN,NaN,NaN,NaN,NaN,geo,NaN
3,g77_and_oecd_countries,G77 and OECD countries,entity_set,NaN,NaN,NaN,NaN,NaN,NaN,geo,NaN
4,geographic_regions_in_4_colors,Geographic regions in 4 colors,entity_set,NaN,NaN,NaN,NaN,NaN,NaN,geo,NaN
5,main_religion_2008,Main religion 2008,entity_set,NaN,NaN,NaN,NaN,NaN,NaN,geo,NaN
6,geo,Geo,entity_domain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,country,Country,entity_set,NaN,NaN,NaN,NaN,NaN,"[geographic_regions, income_groups, landlocked...",geo,NaN
8,time,Time,time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,name,Name,string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [310]:
meta2 = json.load(open('./source/metadata.json'))

In [312]:
meta2['indicatorsDB']['population']

{'domain': [14000, 1400000000],
 'scaleType': 'log',
 'scales': ['linear', 'log'],
 'sourceLink': 'http://www.gapminder.org/news/data-sources-dont-panic-end-poverty',
 'use': 'indicator'}

In [361]:
cc2 = pd.DataFrame([], columns=cc.columns)

cc2.concept = rm.values()

In [362]:
cc2['name'] = cc2.concept
cc2['concept_type'] = 'measure'

cc2['indicator_url'] = cc2['concept'].apply(lambda x: meta2['indicatorsDB'][x[3:]]['sourceLink'])

cc2['scales'] = cc2['concept'].apply(lambda x: meta2['indicatorsDB'][x[3:]]['scales'])


In [363]:
cc2

,concept,name,concept_type,tooltip,indicator_url,scale,unit,interpolation,drill_up,domain,scales
0,sg_population,sg_population,measure,NaN,http://www.gapminder.org/news/data-sources-don...,NaN,NaN,NaN,NaN,NaN,"[linear, log]"
1,sg_gini,sg_gini,measure,NaN,http://www.gapminder.org/news/data-sources-don...,NaN,NaN,NaN,NaN,NaN,[linear]
2,sg_gdp_p_cap_const_ppp2011_dollar,sg_gdp_p_cap_const_ppp2011_dollar,measure,NaN,http://www.gapminder.org/news/data-sources-don...,NaN,NaN,NaN,NaN,NaN,"[log, linear]"


In [368]:
c_all = pd.concat([dc, cc, cc2])
c_all = c_all.rename(columns={'tooltip': 'description'})

In [546]:
c_all.iloc[42]['name'] = 'Scales'
c_all.iloc[42]['concept'] = 'scales'

In [541]:
c_all = c_all.drop('scale', axis=1)

In [568]:
c_all = c_all.drop([14, 15, 16, 17, 18, 19])

In [602]:
c_all = c_all.drop([40])

In [603]:
c_all.to_csv('./output/ddf/ddf--concepts.csv', index=False)

# data points

In [60]:
path = '../old/data/out/gw/indicators/'

In [61]:
fs = os.listdir(path)

In [62]:
idt[idt['-t-ind'] == '0AgogXXPMARyldC1rcTI5OU50Mnc1djdkNXpnWUFrZmc']['-t-name'].iloc[0]

'Murdered children 0-14 (per 100,000 people)'

In [72]:
rename_col('0AgogXXPMARyldC1rcTI5OU50Mnc1djdkNXpnWUFrZmc')

'murdered_children_0_14_per_100000_people'

In [ ]:
# now try to make data points

In [75]:
for f in fs[1:]:
    p = os.path.join(path, f)
    
    col = f[:-5]
    col_r = rename_col(col)
  
    d = pd.read_json(p)

    if 'geo' in d.columns:
        d['geo'] = d['geo'].apply(rename_geo)
        d = d.rename(columns={col:col_r}, index=m)  # MODI
        d.to_csv('./output/ddf/ddf--datapoints--'+col_r+'--by--geo--time.csv', index=False)
    else:
        print(col, ':', col_r)

rj8WsGGNXkijP447qnY5RLg : ———————————————————————
rMF2H57_Vd8TdobvtNhYuwg : ———————————————————————
tAwlNo30o6vVgDUk0EmDolQ : ———————————————————————
tDkfgIt5oQ_ZJQ7IBmUvsUg : ———————————————————————
tdKPTXS1U5Jn3qAQICk4-Vw : ———————————————————————
teiT4rPfnWcd5mCldIm4Zhw : ———————————————————————
tFc2tyrcWLcUyGhq6mTAbDg : ———————————————————————
tFrUv6vnhhO4Jr1czdG_p0Q : ———————————————————————
tIL264Nlw0AlIWlXuCWWNmQ : ———————————————————————
tJwP0x54AZ-X-bGL-RebrpA : ———————————————————————
tLiJLI_rimqrdQRWdamPPRQ : ———————————————————————
tnDimTzGwU_BbmtOMHOUFFQ : ———————————————————————
ts7TQEIg0amXovCDzQxV5EA : ———————————————————————
tYvQLeBTuoKsyofvwGqS-9w : ———————————————————————


In [106]:
len(idt)

542

In [107]:
len(concepts)

544

---

# Other Todos

In [106]:
en = open('/Users/semio/src/work/Gapminder/vizabi/.data/translation/en.json')
enj = json.load(en)

In [107]:
enj

{'buttons/about': 'About',
 'buttons/axes': 'X and Y',
 'buttons/axesmc': 'X and Y',
 'buttons/axis_x': 'X axis',
 'buttons/axis_y': 'Y axis',
 'buttons/color': 'Color',
 'buttons/colors': 'Color',
 'buttons/deselect': 'Deselect',
 'buttons/desktop': 'Desktop',
 'buttons/expand': 'Expand',
 'buttons/find': 'Find',
 'buttons/label': 'Labels',
 'buttons/lock': 'Lock',
 'buttons/more_options': 'Options',
 'buttons/ok': 'OK',
 'buttons/opacity': 'Opacity',
 'buttons/opacityNonselect': 'Opacity of non-selected',
 'buttons/opacityRegular': 'Regular opacity',
 'buttons/presentation': 'Present',
 'buttons/show': 'Show',
 'buttons/showall': 'Show all',
 'buttons/size': 'Size',
 'buttons/size_label': 'Size',
 'buttons/speed': 'Speed',
 'buttons/stack': 'Stack',
 'buttons/trails': 'Trails',
 'buttons/unexpand': 'Restore',
 'check/adaptMinMaxZoom': 'Follow bubbles with zoom',
 'check/presentation': 'Increase text size for presentation',
 'check/removeLabelBox': 'Remove label boxes',
 'hints/bubbl/

In [126]:
concept_file = './source/graph_settings - Indicators.csv'

concepts = pd.read_csv(concept_file)
k = concepts[concepts['ddf_id'] == u'———————————————————————'].index
concepts = concepts.drop(k)

concepts.columns = list(map(to_concept_id, concepts.columns))

trs = concepts[['ddf_id', 'ddf_name', 'ddf_unit', 'tooltip']].copy()
trs['key_1'] = 'indicator/'+trs['ddf_id']
trs['key_2'] = 'unit/'+trs['ddf_id']
trs['key_3'] = 'description/'+trs['ddf_id']

trs1 = trs.drop('ddf_id', axis=1).set_index('key_1')
trs2 = trs.drop('ddf_id', axis=1).set_index('key_2')
trs3 = trs.drop('ddf_id', axis=1).set_index('key_3')

enj.update(trs1['ddf_name'].dropna().to_dict())
enj.update(trs2['ddf_unit'].dropna().to_dict())
enj.update(trs3['tooltip'].dropna().to_dict())

In [115]:
with open('./output/en.json', 'w') as f:
    json.dump(enj, f, indent=1, sort_keys=True)
    f.close()

In [121]:
# old

trs = concepts[['ddf_id', 'ddf_name', 'ddf_unit', 'tooltip', 'old_ddf_id']].copy()
a = trs[['ddf_id', 'old_ddf_id']].ix[trs['old_ddf_id'].dropna().index].set_index('ddf_id')['old_ddf_id']

In [127]:
for k, v in a.iteritems():
    
    for i in ['indicator/', 'unit/', 'description/']:
        try:
            d = enj[i+k]
            del enj[i+k]
            enj[i+v] = d
        except KeyError:
            continue

In [128]:
with open('./output/en_old.json', 'w') as f:
    json.dump(enj, f, indent=1, sort_keys=True)
    f.close()

In [39]:
indb_file = '/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata.json'
indb_0 = json.load(open(indb_file))

In [4]:
indb_0

{'indicatorsDB': {'20120905_extreme_poverty_percent_people_below_125_a_day': {'availability': [1978,
    2011],
   'domain': [0.0, 92.55],
   'scales': ['linear'],
   'sourcelink': 'https://docs.google.com/spreadsheet/pub?key=0ArfEDsV3bBwCdDhjcXdjbURLMFFVcVFPYThhYmtvZXc',
   'use': 'indicator'},
  '_default': {'scales': ['ordinal'], 'sourceLink': '', 'use': 'constant'},
  'adults_with_hiv_percent_age_15_49': {'availability': [1979, 2011],
   'domain': [0.00951, 26.5],
   'scales': ['log'],
   'sourcelink': 'http://spreadsheets.google.com/pub?key=pyj6tScZqmEfbZyl0qjbiRQ',
   'use': 'indicator'},
  'age': {'scales': ['linear', 'log'], 'sourceLink': '', 'use': 'indicator'},
  'age_at_1st_marriage_women': {'availability': [1616, 2005],
   'domain': [12.5, 33.66],
   'scales': ['linear'],
   'sourcelink': 'http://spreadsheets.google.com/pub?key=t4eF8H_jq_xyKCUHAX6VT1g',
   'use': 'indicator'},
  'aged_15_24_employment_rate_percent': {'availability': [1991, 2007],
   'domain': [11.3, 81.2],


In [41]:
indb_0['indicatorsTree']

{'children': [{'id': 'time'},
  {'children': [{'id': 'geo'}, {'id': 'geo.name'}, {'id': 'geo.region'}],
   'id': '_properties'},
  {'children': [{'children': [{'id': 'gini'}], 'id': 'inequality'},
    {'id': 'gdp_p_cap_const_ppp2011_dollar'},
    {'id': 'gdp_const_ppp2011_dollar'},
    {'children': [{'id': 'aid_given_2007_us'},
      {'id': 'aid_given_per_person_2007_us'},
      {'id': 'aid_given_percent_of_gni'},
      {'id': 'water_and_sanitation_aid_given_percent_of_aid'},
      {'id': 'economical_infrastructure_aid_given_percent_of_aid'},
      {'id': 'production_sector_aid_given_percent_of_aid'},
      {'id': 'multisector_cross_cutting_aid_given_percent_of_aid'},
      {'id': 'education_aid_given_percent_of_aid'},
      {'id': 'health_aid_given_percent_of_aid'},
      {'id': 'population_policies_aid_given_percent_of_aid'},
      {'id': 'government_and_society_aid_given_percent_of_aid'},
      {'id': 'other_social_services_aid_given_percent_of_aid'}],
     'id': 'aid_given'},
    {

In [33]:
# insert menu

def get_menus(tree, upper=''):
    res = []
    if upper:
        lv1 = get_menus(tree)
        i = lv1.index(upper)
        if 'children' in tree['children'][i].keys():
            for t in tree['children'][i]['children']:
                res.append(t['id'])
    else:       
        for t in tree['children']:
            res.append(t['id'])
    
    return res


In [48]:
get_menus(indb_0['indicatorsTree'])

['time',
 '_properties',
 'economy',
 'health',
 'population',
 '_default',
 'children_per_woman_total_fertility',
 'co2_emissions_tonnes_per_person',
 'income_per_person_gdppercapita_ppp_inflation_adjusted',
 'child_mortality_0_5_year_olds_dying_per_1000_born',
 'life_expectancy_years',
 'education',
 'for_advanced_users',
 'work',
 'population',
 'infrastructure',
 'energy',
 'environment',
 'society',
 'lol',
 'testing_root']

In [28]:
concepts.columns = list(map(to_concept_id, concepts.columns))
concepts['menu_level1'] = concepts['menu_level1'].apply(to_concept_id)
concepts['menu_level_2'] = concepts['menu_level_2'].apply(to_concept_id)

k = concepts[concepts['ddf_id'] == u'———————————————————————'].index
concepts = concepts.drop(k)

In [30]:
g = concepts.set_index('ddf_id').groupby(['menu_level1', 'menu_level_2']).groups

In [43]:
g2 = {('economy', np.nan): ['testing'],
      (np.nan, np.nan): ['testing_root'],
      ('economy', 'newm'): ['ohnew'],
      ('lol', np.nan): ['lololol'],
      ('lol', 'ohohoh'): ['ohsjkd', 'jisodjfojs'],
     }

In [45]:
# TODO: filter it first from indicatorDB

for k, v in g2.items():
    
    l1, l2 = k
    
    if l1 is np.nan:
        for c in v:
            d = {'id': c}
            if d not in indb_0['indicatorsTree']['children']:
                indb_0['indicatorsTree']['children'].append(d)
    
    else:
        try:
            m1 = get_menus(indb_0['indicatorsTree'])
            i1 = m1.index(l1)
        except ValueError:
            indb_0['indicatorsTree']['children'].append({'id': l1, 'children': []})
            m1 = get_menus(indb_0['indicatorsTree'])
            i1 = m1.index(l1)
        
        if l2 is np.nan:
            for c in v:
                d = {'id': c}
                if d not in indb_0['indicatorsTree']['children'][i1]['children']:
                    indb_0['indicatorsTree']['children'][i1]['children'].append(d)
                    
        else:
            try:
                m2 = get_menus(indb_0['indicatorsTree'], l1)
                i2 = m2.index(l2)
            except ValueError:
                if 'children' in indb_0['indicatorsTree']['children'][i1].keys():
                    indb_0['indicatorsTree']['children'][i1]['children'].append({'id': l2, 'children': []})
                else:
                    indb_0['indicatorsTree']['children'][i1]['children'] = [{'id': l2, 'children': []}]
                m2 = get_menus(indb_0['indicatorsTree'], l1)
                i2 = m2.index(l2)
            
            for c in v:
                d = {'id': c}
                if d not in indb_0['indicatorsTree']['children'][i1]['children'][i2]['children']:              
                    indb_0['indicatorsTree']['children'][i1]['children'][i2]['children'].append(d)

        

In [51]:
i1 = get_menus(indb_0['indicatorsTree']).index('lol')

indb_0['indicatorsTree']['children'][i1]

{'children': [{'id': 'lololol'},
  {'children': [{'id': 'ohsjkd'}, {'id': 'jisodjfojs'}], 'id': 'ohohoh'}],
 'id': 'lol'}

In [131]:
for i in concepts['ddf_id']:
    if i not in indb_0['indicatorsDB'].keys():
        print(i)

In [139]:
# also applies for en.json
f_in = open('/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata.json')
f_out = open('/Users/semio/src/work/Gapminder/vizabi/.data/waffles/metadata_old.json', 'w')

new = f_in.read()

for k, v in a.iteritems():
    if k in new:
        new = new.replace(k, v)
    else:
        print(k)
    
f_out.write(new)
f_out.close()

# 3.22 updates

In [56]:
# 1. Map the geo entity id’s to new ones according to this file: 
# https://github.com/Gapminder/waffle-server/blob/development/csv_data_mapping_cli/geo-mapping.json
import json
mappins = './source/geo-mapping.json'
m = json.load(open(mappins))

# now we go back to the datapoints and entities. check above 

In [299]:
mm = country4[['country', 'country_2']]

In [303]:
mmm = mm[mm.country != mm.country_2].set_index('country_2')['country'].to_dict()

In [306]:
json.dump(mmm, open('./output/tmp/geo-mapping.json', 'w'), indent=1)

In [179]:
# new data points
dpc = pd.read_csv('./source/dont-panic-poverty.csv')

In [181]:
rm = {'gini': 'sg_gini', 
      'population': 'sg_population',
      'gdp_p_cap_const_ppp2011_dollar': 'sg_gdp_p_cap_const_ppp2011_dollar'
     }

In [184]:
dpc = dpc.rename(columns=rm)  # TODO: add to concept file.

In [189]:
for k in rm.values():
    dpc[['geo', 'time', k]].dropna().sort_values(by=['geo', 'time']).to_csv('./output/ddf/ddf--datapoints--'+k+'--by--geo--time.csv', index=False)

In [370]:
# metadata.json

c_all2 = pd.concat([cc, cc2])

mdata = c_all2[['concept', 'indicator_url', 'scales', 'interpolation']]

In [371]:
mdata.head()

,concept,indicator_url,scales,interpolation
0,children_per_woman_total_fertility,http://spreadsheets.google.com/pub?key=phAwcNA...,"[linear, log]",NaN
1,co2_emissions_tonnes_per_person,http://spreadsheets.google.com/pub?key=phAwcNA...,"[linear, log]",NaN
2,income_per_person_gdppercapita_ppp_inflation_a...,http://spreadsheets.google.com/pub?key=phAwcNA...,"[log, linear]",NaN
3,child_mortality_0_5_year_olds_dying_per_1000_born,http://spreadsheets.google.com/pub?key=0ArfEDs...,"[log, linear]",NaN
4,life_expectancy_years,http://spreadsheets.google.com/pub?key=phAwcNA...,"[linear, log]",NaN


In [372]:
mdata = mdata.set_index('concept')

In [373]:
mdata.columns = ['sourceLink', 'scales', 'interpolation']

In [374]:
mdata['use'] = 'indicator'

In [375]:
mdata.head()

,sourceLink,scales,interpolation,use
concept,,,,
children_per_woman_total_fertility,http://spreadsheets.google.com/pub?key=phAwcNA...,"[linear, log]",NaN,indicator
co2_emissions_tonnes_per_person,http://spreadsheets.google.com/pub?key=phAwcNA...,"[linear, log]",NaN,indicator
income_per_person_gdppercapita_ppp_inflation_adjusted,http://spreadsheets.google.com/pub?key=phAwcNA...,"[log, linear]",NaN,indicator
child_mortality_0_5_year_olds_dying_per_1000_born,http://spreadsheets.google.com/pub?key=0ArfEDs...,"[log, linear]",NaN,indicator
life_expectancy_years,http://spreadsheets.google.com/pub?key=phAwcNA...,"[linear, log]",NaN,indicator


In [604]:
indb = {'indicatorsDB': {}}

In [605]:
indb['indicatorsDB'].update(to_dict_dropna(mdata))

In [606]:
panic = dict([[rm[i],meta2['indicatorsDB'][i]] for i in rm.keys()])

In [607]:
indb['indicatorsDB'].update(panic)

In [608]:
# geo.world_4region
geomd = {'geo.world_4region': meta2['indicatorsDB']['geo.region']}

In [609]:
indb['indicatorsDB'].update(geomd)

In [382]:
# area
area = json.load(open('./source/area_categorizarion.json'))

In [626]:
res = {}

for i in range(len(area)):
    key = to_concept_id(area[i]['n'])
    source = area[i]['sourceName']
    
    res['geo.'+key] = {'use': 'property', 'scales': ['ordinal'], 'sourceLink': source}

In [627]:
indb['indicatorsDB'].update(res)

In [636]:
meta2['indicatorsDB']['population']

{'availability': [1800, 2015],
 'domain': [50.0, 1376048943.0],
 'scaleType': 'log',
 'scales': ['linear', 'log'],
 'sourceLink': 'http://www.gapminder.org/news/data-sources-dont-panic-end-poverty',
 'use': 'indicator'}

In [387]:
area[1].keys()

dict_keys(['dataCollectionUrl', 'groupings', 'id', 'n', 'a', 'sourceName', 'providerUrl'])

In [391]:
area[1]['a']

'How rich are you?'

In [332]:
indb['indicatorsDB']

{'20120905_extreme_poverty_percent_people_below_125_a_day': {'scales': ['linear',
   'log'],
  'sourceLink': 'https://docs.google.com/spreadsheet/pub?key=0ArfEDsV3bBwCdDhjcXdjbURLMFFVcVFPYThhYmtvZXc',
  'use': 'indicator'},
 'adults_with_hiv_percent_age_15_49': {'scales': ['log', 'linear'],
  'sourceLink': 'http://spreadsheets.google.com/pub?key=pyj6tScZqmEfbZyl0qjbiRQ',
  'use': 'indicator'},
 'age_at_1st_marriage_women': {'scales': ['linear', 'log'],
  'sourceLink': 'http://spreadsheets.google.com/pub?key=t4eF8H_jq_xyKCUHAX6VT1g',
  'use': 'indicator'},
 'aged_15_24_employment_rate_percent': {'scales': ['linear', 'log'],
  'sourceLink': 'http://spreadsheets.google.com/pub?key=rfHz_nx27dDQo4dUoIeVT3A',
  'use': 'indicator'},
 'aged_15_24_unemployment_rate_percent': {'scales': ['linear', 'log'],
  'sourceLink': 'http://spreadsheets.google.com/pub?key=rb0oP4d1BREXa8xMIUf4NZg',
  'use': 'indicator'},
 'aged_15_64_labour_force_participation_rate_percent': {'scales': ['linear',
   'log'],


In [610]:
for i in indb['indicatorsDB'].keys():
    fname = os.path.join('./output/ddf/', 'ddf--datapoints--'+i+'--by--geo--time.csv')
    try:
        df = pd.read_csv(fname, dtype={i:float, 'time': int})
    except OSError:
        print('no data for', i)
        continue
        
    dm = [float(df[i].min()), float(df[i].max())]
    av = [int(df['time'].min()), int(df['time'].max())]

    indb['indicatorsDB'][i].update({'domain': dm, 'availability': av})

no data for geo.world_4region
no data for newborn_mortality_rate_per_1000
no data for newborn_deaths


In [399]:
from collections import OrderedDict

In [621]:
# indb tree
indb['indicatorsTree'] = OrderedDict([['id', '_root'], ['children', []]])

In [622]:
ti = OrderedDict([['id', 'time']])
pro = OrderedDict([['id', '_properties'], ['children', [{'id': 'geo'}, {'id': 'geo.name'}]]])

In [623]:
indb['indicatorsTree']['children'].append(ti)


In [449]:
c_all3 = concepts[['concept', 'menu_level1', 'menu_level_2']].sort_values(['menu_level1', 'menu_level_2'], na_position='first')

In [450]:
c_all3 = c_all3.drop(k)

In [451]:
c_all3['menu_level1'] = c_all3['menu_level1'].apply(to_concept_id).fillna('0')
c_all3['menu_level_2'] = c_all3['menu_level_2'].apply(to_concept_id).fillna('1')

In [452]:
c_all3.set_index('concept').groupby('menu_level1').groups

{'0': ['children_per_woman_total_fertility',
  'co2_emissions_tonnes_per_person',
  'income_per_person_gdppercapita_ppp_inflation_adjusted',
  'child_mortality_0_5_year_olds_dying_per_1000_born',
  'life_expectancy_years'],
 'economy': ['aid_given_2007_us',
  'aid_given_per_person_2007_us',
  'aid_given_percent_of_gni',
  'water_and_sanitation_aid_given_percent_of_aid',
  'economical_infrastructure_aid_given_percent_of_aid',
  'production_sector_aid_given_percent_of_aid',
  'multisector_cross_cutting_aid_given_percent_of_aid',
  'education_aid_given_percent_of_aid',
  'health_aid_given_percent_of_aid',
  'population_policies_aid_given_percent_of_aid',
  'government_and_society_aid_given_percent_of_aid',
  'other_social_services_aid_given_percent_of_aid',
  'aid_received_total_us_inflation_adjusted',
  'aid_received_per_person_current_us',
  'aid_received_percent_of_gni',
  'debt_to_foreigners_by_public_and_private_percent_of_gni',
  'total_reserves_percent_of_debt_to_foreigners',
  'ex

In [460]:
c_all3 = c_all3.set_index('concept')

In [481]:
c_all3.ix[c_all3.groupby('menu_level1').groups['for_advanced_users']].groupby('menu_level_2').groups

{'1': ['ifpri_underweight_children'],
 'advanced_debt_trade': ['debt_servicing_costs_percent_of_exports_and_net_income_from_abroad',
  'external_debt_total_us_not_inflation_adjusted',
  'present_value_of_debt_percent_of_gni',
  'exports_unit_value_index_2000100',
  'imports_unit_value_index_2000100',
  'net_barter_terms_of_trade_2000_100'],
 'alternative_child_mortality_data': ['under_five_mortality_from_cme_per_1000_born',
  'under_five_mortality_from_ihme_per_1000_born',
  'child_mortality_0_5_year_olds_more_years_version_7'],
 'alternative_demography_indicators': ['life_expectancy_at_birth_data_from_ihme',
  'children_per_woman_temporary_update'],
 'alternative_energy': ['energy_supply_per_person_toe',
  'energy_from_solid_biofuels_percent',
  'residential_energy_use_percent'],
 'alternative_gdp_data': ['gdppercapita_growth_percent_per_year',
  'old_version_of_income_per_person_version_3',
  'old_version_of_income_per_person_version_8',
  'alternative_gdppercapita_ppp_inflation_adju

In [468]:
g = c_all3.groupby('menu_level1').groups

In [471]:
list(sorted(g.keys()))

['0',
 'economy',
 'education',
 'energy',
 'environment',
 'for_advanced_users',
 'health',
 'infrastructure',
 'population',
 'society',
 'work']

In [624]:
g = c_all3.groupby('menu_level1').groups

ks = list(sorted(g.keys()))

if 'for_advanced_users' in ks:
    ks.remove('for_advanced_users')
    ks.append('for_advanced_users')    
    
for key in ks:
    if key == '0':
        for i in sorted(g[key]):
            indb['indicatorsTree']['children'].append({'id': i})
        indb['indicatorsTree']['children'].append(pro)
        for i in range(len(area)):
            key = 'geo.'+to_concept_id(area[i]['n'])
            indb['indicatorsTree']['children'][-1]['children'].append({'id': key})
        indb['indicatorsTree']['children'][-1]['children'].append({'id': 'geo.world_4region'})
        continue
    
    od = OrderedDict([['id', key], ['children', []]])
    indb['indicatorsTree']['children'].append(od)
    
    g2 = c_all3.ix[g[key]].groupby('menu_level_2').groups
    for key2 in sorted(g2.keys()):
        if key2 == '1':
            for i in sorted(g2[key2]):
                indb['indicatorsTree']['children'][-1]['children'].append({'id': i})
        else:
            od = OrderedDict([['id', key2], ['children', []]])
            indb['indicatorsTree']['children'][-1]['children'].append(od)
            for i in sorted(g2[key2]):
                indb['indicatorsTree']['children'][-1]['children'][-1]['children'].append({'id': i})

In [ ]:
print(json.dumps(indb['indicatorsTree'], indent=1))

In [628]:
with open('./output/vizabi/metadata.json', 'w') as f:
    json.dump(indb, f, indent=1)
    f.close()

In [676]:
# en.json

enj = json.load(open('./source/en.json'))

In [505]:
enj['unit/']  #?

''

In [645]:
trs = c_all[['concept', 'name', 'unit', 'description', 'domain']].copy()

In [646]:
trs = trs.iloc[6:-3].copy()

In [641]:
trs

,concept,name,unit,description,domain
12,name_long,Name Long,NaN,NaN,NaN
13,description,Description,NaN,NaN,NaN
20,gapminder_list,gapminder_list,NaN,NaN,NaN
21,alternative_1,alternative_1,NaN,NaN,NaN
22,alternative_2,alternative_2,NaN,NaN,NaN
23,alternative_3,alternative_3,NaN,NaN,NaN
24,alternative_4_cdiac,alternative_4_cdiac,NaN,NaN,NaN
25,pandg,pandg,NaN,NaN,NaN
26,god_id,god_id,NaN,NaN,NaN
27,alt_5,alt_5,NaN,NaN,NaN


In [677]:
trs['key_1'] = 'indicator/'+trs['concept']
trs['key_2'] = 'unit/'+trs['concept']
trs['key_3'] = 'description/'+trs['concept']

trs1 = trs.drop('concept', axis=1).set_index('key_1')
trs2 = trs.drop('concept', axis=1).set_index('key_2')
trs3 = trs.drop('concept', axis=1).set_index('key_3')

enj.update(trs1['name'].dropna().to_dict())
enj.update(trs2['unit'].dropna().to_dict())
enj.update(trs3['description'].dropna().to_dict())

In [589]:
c_all4 = concepts[['concept', 'menu_level1', 'menu_level_2']]

l1 = c_all4['menu_level1'].unique()
l2 = c_all4['menu_level_2'].unique()

In [678]:
for i in l1:
    if i is not np.nan:
        key = to_concept_id(i)
        enj['indicator/'+key] = i
    
for i in l2:
    if i is not np.nan:
        key = to_concept_id(i)
        enj['indicator/'+key] = i

In [679]:
enj['indicator/aid_given']

'Aid given'

In [597]:
c5 = c_all[c_all['concept_type'] == 'entity_set'].copy()

In [598]:
c5.loc[37, 'name'] = 'World Regions'  # TODO read from file

In [672]:
for i, v in c5.iterrows():
    enj['indicator/'+'geo.'+v['concept']] = v['name']

In [673]:
enj['indicator/sg_population'] = enj['indicator/population']
enj['indicator/sg_gini'] = enj['indicator/gini']
enj['indicator/sg_gdp_p_cap_const_ppp2011_dollar'] = enj['indicator/gdp_p_cap_const_ppp2011_dollar']

In [674]:
enj['unit/sg_population'] = enj['unit/population']
enj['unit/sg_gini'] = enj['unit/gini']
enj['unit/sg_gdp_p_cap_const_ppp2011_dollar'] = enj['unit/gdp_p_cap_const_ppp2011_dollar']

In [ ]:
#enj['indicator/sg_population'] = enj.pop('indicator/population')
#enj['indicator/sg_gini'] = enj.pop('indicator/gini')
#enj['indicator/sg_gdp_p_cap_const_ppp2011_dollar'] = enj.pop('indicator/gdp_p_cap_const_ppp2011_dollar')

In [675]:
with open('./output/vizabi/en.json', 'w') as f:
    json.dump(enj, f, indent=1, sort_keys=True)
    f.close()
